# Setup and API Key and Configuration
Install the elsapy library and configure the API key required for authentication with Elsevier's APIs.

In [21]:
# Import necessary libraries
import json
import csv

from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch

In [22]:
# Load API key from a configuration file
with open("config.json") as config_file:
    config = json.load(config_file)

# Initialize the ElsClient with the API key
client = ElsClient(config['apikey'])

# Initialize the Elsevier Client
Create an Elsevier client instance using your API key and configure the connection settings.

In [23]:
# Set the connection settings for the client
client.base_url = 'https://api.elsevier.com/content/search/scopus'

# Verify the client connection
if client:
    print("Client initialized successfully.")
else:
    print("Failed to initialize client.")

Client initialized successfully.


# Search for Articles by Keyword
Demonstrate how to search for articles in Scopus using keywords and how to handle the response data.

In [26]:
# Import necessary libraries for searching articles

# Define the search query using a keyword
search_query = (
    'TITLE-ABS-KEY('
    '("smart cit*" OR "intelligent cit*" OR "digital city" OR '
    '"smart urban" OR "urban computing") '
    "AND "
    '("generative AI" OR "generative artificial intelligence" OR '
    '"GAN" OR "generative adversarial network*" OR "diffusion model*" OR '
    '"large language model*" OR "LLM" OR "LLMs" OR '
    '"variational autoencoder*" OR "VAE*" OR "autoregressive model*" OR '
    '"transformer*" OR "retrieval-augmented generation" OR "RAG" OR "AI agent*")'
    ") "
    "AND PUBYEAR > 2018 AND PUBYEAR < 2026 "
    'AND LIMIT-TO(LANGUAGE, "English")'
)

# Initialize the ElsSearch object with the search query and client
doc_srch = ElsSearch(search_query, 'scopus')

# Execute the search
doc_srch.execute(client, get_all=True)

# Check if the search was successful
if doc_srch.results:
    print(f"Found {len(doc_srch.results)} results.")
else:
    print("No results found.")

Found 600 results.


# Save the results to a CSV file

In [27]:
# Function to export results to CSV with dynamic fields
def export_to_csv(data, filename):
    # Get all possible keys across all dictionaries
    all_keys = set()
    for item in data:
        all_keys.update(item.keys())

    # Convert to sorted list for consistent column order
    fieldnames = sorted(list(all_keys))

    with open(filename, "w", newline="", encoding="utf-8") as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        dict_writer.writeheader()
        dict_writer.writerows(data)

    print(f"CSV exported {len(data)} records")


# Function to export results to JSON
def export_to_json(data, filename):
    with open(filename, "w", encoding="utf-8") as output_file:
        json.dump(data, output_file, indent=4, ensure_ascii=False)

    print(f"JSON exported with {len(data)} records")

In [28]:
# Filter out documents with missing doi
results = [doc for doc in doc_srch.results if doc.get("prism:doi")]

# Export the results to a CSV file
export_to_csv(results, "../data/01_scopus_results.csv")
export_to_json(results, "../data/01_scopus_results.json")

CSV exported 512 records
JSON exported with 512 records
